## Setup

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
import pandas as pd

df = pd.read_csv("hf://datasets/sakren/twitter_racism_dataset/twitter_racism_parsed_dataset.csv")


# Tentukan path folder 'data' di direktori saat ini
save_dir = "data"
os.makedirs(save_dir, exist_ok=True)  # Buat folder jika belum ada


In [3]:
df.head()

,index,id,Text,Annotation,oh_label
0,5.767493e+17,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.408905e+17,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.678433e+17,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.766462e+17,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.713492e+17,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [4]:
# Remove unused features
df = df.drop(columns=['index', 'id', 'Annotation'])

# Simpan file CSV dalam folder 'data'
save_path = os.path.join(save_dir, "data.csv")
df.to_csv(save_path, index=False)

print(f"File disimpan di {save_path}")

File disimpan di data\data.csv


In [5]:
PIPELINE_NAME = "racism-pipeline"
SCHEMA_PIPELINE_NAME = "racism-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where createdmodels from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = "data"

os.makedirs("module", exist_ok=True)

In [6]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

## Data Ingestion

In [7]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [8]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation
### 1. Summary Statistic

In [9]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples']
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(statistics_gen.outputs["statistics"])

### 2. Data Schema

In [11]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Text',BYTES,required,,-
'oh_label',INT,required,,-


### 3. Identifikasi Anomali

In [13]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(example_validator.outputs["anomalies"])

## Data Preprocessing

In [15]:
TRANSFORM_MODULE_FILE = 'module/data_transform.py'

In [16]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = 'oh_label'
FEATURE_KEY = 'Text'
def transformed_name(key):
    return key + "_xf"

def preprocessing_fn(inputs):
    
    outputs = {}
    # Pastikan teks dalam bentuk DenseTensor sebelum transformasi
    text_input = inputs[FEATURE_KEY]
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(text_input)
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    return outputs

Writing module/data_transform.py


In [17]:
transform = Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file = TRANSFORM_MODULE_FILE,
)

interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\15538f4a8c644afd8a13c189eed893f3\assets


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\15538f4a8c644afd8a13c189eed893f3\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Pengembangan Model

### Tuner

In [18]:
TUNER_MODULE_FILE = "module/tuner.py"

In [19]:
%%writefile {TUNER_MODULE_FILE}

from typing import Any, Dict, NamedTuple, Text
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow.keras.layers import LeakyReLU

LABEL_KEY = 'oh_label'
FEATURE_KEY = 'Text'
VOCAB_SIZE = 10000
epochs = 5

TunerFnResult = NamedTuple(
    "TunerFnResult", [("tuner", base_tuner.BaseTuner), ("fit_kwargs", Dict[Text, Any])]
)

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(
    file_pattern, 
    tf_transform_output, 
    num_epochs, 
    batch_size=64) -> tf.data.Dataset:

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create bacthes of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size = batch_size,
        features = transform_feature_spec,
        reader = gzip_reader_fn,
        num_epochs = num_epochs,
        label_key = transformed_name(LABEL_KEY)
    ).repeat(epochs)

    return dataset
 

def model_builder(hp, vectorizer):
    ### Define parameter yang digunakan untuk tuning
    num_layer = hp.Int("num_layer", min_value=1, max_value=5, step=1)
    embed_dim = hp.Int("embed_dim", min_value=16, max_value=128, step=32)
    fc_layer = hp.Int("fc_layer", min_value=32, max_value=64, step=16)
    lr = hp.Choice("lr", values=[1e-2, 1e-3, 1e-4])
    # lstm_units = hp.Int("lstm_units", min_value=32, max_value=128, step=32)

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorizer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embed_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.2)(x)
    # x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    for _ in range(num_layer):
        x = layers.Dense(fc_layer, activation='relu')(x)
    # x = layers.Dense(fc_layer, activation='relu')(x)
    # x = layers.Dense(fc_layer, activation="relu")(x)
    
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss = tf.keras.losses.BinaryCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

def tuner_fn(fn_args):
    # Membuat training dan validation datasetyang telah di preprocessing
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=epochs)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=epochs)

    vectorize_layer = layers.TextVectorization(
        standardize = "lower_and_strip_punctuation",
        max_tokens = 200,
        output_mode = 'int',
        output_sequence_length = 100
    )

    vectorize_layer.adapt(train_set.map(lambda x, _: x[transformed_name(FEATURE_KEY)]))

    # Callback untuk early stopping
    stop_early = tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy', 
        mode='max', 
        verbose=1, 
        patience=15
    )

    # Mendefinisikan strategi hyperparameter tuning
    # tuner = kt.RandomSearch(
    #     hypermodel = lambda hp: model_builder(hp, vectorize_layer),
    #     objective = 'val_sparse_categorical_accuracy',
    #     max_trials = epochs,
    #     seed = 28,
    # )
    tuner = kt.Hyperband(
        lambda hp: model_builder(hp, vectorize_layer),
        objective = 'val_binary_accuracy',
        max_epochs = epochs,
        factor = 3,
        directory = fn_args.working_dir,
        project_name = 'kt_hyperband'
    )

    return TunerFnResult(
        tuner = tuner,
        fit_kwargs = {
            "callbacks": [stop_early],
            'x' : train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
        }
    )


Writing module/tuner.py


In [20]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2

tuner = Tuner(
    module_file = TUNER_MODULE_FILE,
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args = trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
)

interactive_context.run(tuner)

Trial 10 Complete [00h 00m 37s]
val_binary_accuracy: 0.9207812547683716

Best val_binary_accuracy So Far: 0.9248437285423279
Total elapsed time: 00h 05m 13s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\racism-pipeline\.temp\6\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
num_layer: 3
embed_dim: 112
fc_layer: 48
lr: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9248437285423279
Trial summary
Hyperparameters:
num_layer: 3
embed_dim: 112
fc_layer: 48
lr: 0.001
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9212499856948853
Trial summary
Hyperparameters:
num_layer: 5
embed_dim: 80
fc_layer: 32
lr: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9207812547683716
Trial summary
Hyperparameters:
num_layer: 1
embed_dim: 48
fc_layer: 64
lr: 0.001
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9207812547683716
Trial summary
Hyperparameters:
num_layer: 5
embed_dim: 48
fc_layer: 48
lr: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9203125238418579
Trial summary


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Trainer

In [21]:
TRAINER_MODULE_FILE = "module/indo_tele_sentiment.py"

In [22]:
%%writefile {TRAINER_MODULE_FILE}

import os
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "oh_label"
FEATURE_KEY = "Text"

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(
    file_pattern, 
    tf_transform_output, 
    num_epochs, 
    batch_size=64) -> tf.data.Dataset:

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create bacthes of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size = batch_size,
        features = transform_feature_spec,
        reader = gzip_reader_fn,
        num_epochs = num_epochs,
        label_key = transformed_name(LABEL_KEY)
    )

    return dataset
 
# Vocab size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
embed_dim = 16
epochs = 50

def model_builder(hp, vectorizer):
    """Build ML model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorizer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp.get("embed_dim"), name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.2)(x)
    for _ in range(hp.get("num_layer")):
        x = layers.Dense(hp.get("fc_layer"), activation='relu')(x)
    # x = layers.Dense(64, activation='relu')(x)
    # x = layers.Dense(32, activation="relu")(x)
    
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss = tf.keras.losses.BinaryCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp.get("lr")),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy', 
        mode='max', 
        verbose=1, 
        patience=10
    )
    mc = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir, 
        monitor='val_binary_accuracy', 
        mode='max', 
        verbose=1, 
        save_best_only=True
    )
    hp = fn_args.hyperparameters.get("values")

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create bacthes of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp.get("tuner/epochs"))
    val_set = input_fn(fn_args.eval_files, tf_transform_output, hp.get("tuner/epochs"))
    vectorize_layer = layers.TextVectorization(
        standardize = "lower_and_strip_punctuation",
        max_tokens = VOCAB_SIZE,
        output_mode = 'int',
        output_sequence_length = SEQUENCE_LENGTH
    )
    vectorize_layer.adapt(train_set.map(lambda x, _: x[transformed_name(FEATURE_KEY)]))


    # Build Model
    model = model_builder(hp, vectorize_layer)

    # Train the model
    model.fit(
        x = train_set.repeat(),
        validation_data = val_set.repeat(),
        callbacks = [tensorboard_callback, es, mc],
        steps_per_epoch = 100,
        validation_steps = 100,
        epochs = epochs
    )

    signatures = {
        'serving_default' : _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
            shape=[None],
            dtype = tf.string,
            name = 'examples'
            )
        )
    }

    model.save(fn_args.serving_model_dir, save_format = 'tf', signatures=signatures)


Writing module/indo_tele_sentiment.py


In [23]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file = os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    hyperparameters= tuner.outputs['best_hyperparameters'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 112)          1120000   
                                                                 
 global_average_pooling1d_1   (None, 112)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 112)               0   

INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Trainer\model\7\Format-Serving\assets


100/100 [==============================] - 6s 48ms/step - loss: 0.3371 - binary_accuracy: 0.8597 - val_loss: 0.1956 - val_binary_accuracy: 0.9171
Epoch 2/50
100/100 [==============================] - ETA: 0s - loss: 0.1821 - binary_accuracy: 0.9300
Epoch 2: val_binary_accuracy improved from 0.91714 to 0.92767, saving model to pipelines\racism-pipeline\Trainer\model\7\Format-Serving


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Trainer\model\7\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\racism-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Analisis dan Validasi Model 

In [24]:
# Melakukan analisis dan validasi dengan menyediakan baseline model menggunakan Resolver

from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [31]:
# Konfigurasi untuk mengevaluasi model

import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(label_key='oh_label')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                              threshold = tfma.MetricThreshold(
                                  value_threshold = tfma.GenericValueThreshold(
                                      lower_bound = {'value':0.5}
                                  ),
                                  change_threshold = tfma.GenericChangeThreshold(
                                      direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                                      absolute={'value': 0.0001}
                                  )
                              ))
        ])
    ]
)

In [32]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config = eval_config
)

interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [33]:
# Visualisasi hasil evaluasi
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Model Deployment

In [28]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model= trainer.outputs['model'],
    model_blessing= evaluator.outputs['blessing'],
    push_destination= pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = 'serving_model_dir/racism-detection-model'
        )
    )
)
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [30]:
import requests
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/racism-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1741734111'}]}
